## Image to Frames

In [90]:
%matplotlib inline
import io
import base64
from IPython.display import HTML
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
import os
# video = io.open('test.avi', 'r+b').read()
FOLDER = "data/"
# cv2.imshow(video)

In [97]:
with open(FOLDER+"train1.txt") as f:
    content = f.readlines()
train = [x.strip().split(" ") for x in content] 
with open(FOLDER+"validation1.txt") as f:
    content = f.readlines()
validation = [x.strip().split(" ") for x in content] 

with open(FOLDER+"trainlist01.txt") as f:
    content = f.readlines()
all_data = [x.strip().split(" ") for x in content] 



In [100]:
filelist = []
for video,label in all_data:
    category = video.split("/")[0]
    filename = video.split("/")[1].split(".avi")[0]    
    directory_name = FOLDER + "UCF101_images/" + category 

    if not os.path.exists(directory_name):
        os.makedirs(directory_name)

    video_file = FOLDER + "UCF101/" + video.split("/")[1]
    cap = cv2.VideoCapture(video_file)
    counter = 0
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % 6) == 0:
            f_ = directory_name + "/" + filename + "_" + str(counter) + ".jpg"
            cv2.imwrite(f_, frame)
            counter += 1
    cap.release()
    filelist.append([category + "/" + filename, counter, label])


## Validation

In [105]:
np.savetxt('data/all_images1.txt', (filelist), fmt=['%s', '%s', '%s'], delimiter=' ')

In [109]:
counter = {i:0 for i in range(101)}
train_counter = {i:0 for i in range(101)}
val_counter = {i:0 for i in range(101)}

with open("data/all_images1.txt") as f:
    for line in f:
        label = int(line.split(" ")[2])
        counter[label-1] += 1        


In [113]:
trainlist = []
validlist = []
with open("data/all_images1.txt") as f:
    for line in f:
        label = int(line.split(" ")[2])
        filename = line.split(" ")[0] + " " + line.split(" ")[1]
        if(val_counter[label-1] < int(0.07 * counter[label-1])):
            val_counter[label-1] += 1            
            validlist.append([filename, label-1])
        else:
            train_counter[label-1] += 1
            trainlist.append([filename, label-1])


In [117]:
np.savetxt('data/train1.txt', (trainlist), fmt=['%s', '%s'], delimiter=' ')
np.savetxt('data/val1.txt', (validlist), fmt=['%s', '%s'], delimiter=' ')